In [24]:
import torch
import torch.nn as nn
import time

# Tạo dữ liệu huấn luyện và phân loại
inputs = torch.randn(100, 784) # tạo dữ liệu huấn luyện với kích thước 100x784
labels = torch.randint(10, (100,)) # tạo dữ liệu phân loại với kích thước 100

# Thiết kế mô hình
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 128) # số lượng đầu vào 784, số lượng đơn vị trong lớp ẩn 128
        self.fc2 = nn.Linear(128, 10) # số lượng đơn vị trong lớp ẩn 128, số lượng đầu ra 10

    def forward(self, x):
        x = torch.relu(self.fc1(x)) # áp dụng hàm kích hoạt ReLU cho lớp ẩn
        x = self.fc2(x)
        return x

# Tạo instance của mô hình
model = Model()

# Huấn luyện mô hình
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

print("Mô hình đã được huấn luyện!")

# Tạo dữ liệu thử nghiệm
test_inputs = torch.randn(100, 784)
test_labels = torch.randint(10, (100,))

# Lập trình đánh giá
model.eval()
with torch.no_grad():
    test_outputs = model(test_inputs)

criterion = nn.CrossEntropyLoss()
test_loss = criterion(test_outputs, test_labels).item()

start_time = time.time()
# Huấn luyện mô hình...
test_outputs = model(test_inputs)
end_time = time.time()

# Đo lường hiệu suất
accuracy = torch.sum(torch.argmax(test_outputs, dim=1) == test_labels).item() / len(test_labels)
print("Độ chính xác:", accuracy * 100)
print("Thời gian huấn luyện:", time.time() - start_time)
print("Thời gian đánh giá:", time.time() - end_time)
print("Thời gian tổng:", time.time() - start_time)

Mô hình đã được huấn luyện!
Độ chính xác: 13.0
Thời gian huấn luyện: 0.0003037452697753906
Thời gian đánh giá: 0.00016307830810546875
Thời gian tổng: 0.00035381317138671875


### Tối ưu ví dụ 5 nhé
- Dùng dữ liệu thật
- Đánh giá đúng thời gian huấn luyện
- Sử dụng kiến trúc mạnh hơn với Dropout để giảm Overfiting.
- Tăng epochs lên 50-100 lần
- Kiểm tra đầu ra, thêm Softmax cho xác suất.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ==== 1. LOAD DATA (Thay vì dùng dữ liệu ngẫu nhiên) ====
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ==== 2. DEFINE MODEL ====
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.3)  # Tránh overfitting

    def forward(self, x):
        x = x.view(-1, 784)  # Chuyển từ 28x28 thành vector 784
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Không dùng Softmax vì CrossEntropyLoss đã xử lý
        return x

# ==== 3. INITIALIZE MODEL, LOSS, AND OPTIMIZER ====
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam tốt hơn SGD ở đây

# ==== 4. TRAIN MODEL ====
n_epochs = 50  # Tăng số epochs để đảm bảo học tốt
train_start_time = time.time()
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
    
    print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
train_end_time = time.time()

print("Mô hình đã được huấn luyện!")

# ==== 5. EVALUATE MODEL ====
model.eval()
test_start_time = time.time()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
test_end_time = time.time()

accuracy = 100 * correct / total

# ==== 6. PRINT RESULTS ====
print(f"Độ chính xác: {accuracy:.2f}%")
print(f"Thời gian huấn luyện: {train_end_time - train_start_time:.4f} giây")
print(f"Thời gian đánh giá: {test_end_time - test_start_time:.4f} giây")
print(f"Thời gian tổng: {train_end_time - train_start_time + test_end_time - test_start_time:.4f} giây")




100%|██████████| 9.91M/9.91M [00:02<00:00, 3.93MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 105kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 241kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.56MB/s]


Epoch [1/50], Loss: 0.4019, Accuracy: 87.48%
Epoch [2/50], Loss: 0.2141, Accuracy: 93.43%
Epoch [3/50], Loss: 0.1746, Accuracy: 94.63%
Epoch [4/50], Loss: 0.1518, Accuracy: 95.28%
Epoch [5/50], Loss: 0.1345, Accuracy: 95.75%
Epoch [6/50], Loss: 0.1248, Accuracy: 96.09%
Epoch [7/50], Loss: 0.1175, Accuracy: 96.36%
Epoch [8/50], Loss: 0.1141, Accuracy: 96.39%
Epoch [9/50], Loss: 0.1078, Accuracy: 96.50%
Epoch [10/50], Loss: 0.1024, Accuracy: 96.67%
Epoch [11/50], Loss: 0.0966, Accuracy: 97.00%
Epoch [12/50], Loss: 0.0911, Accuracy: 97.11%
Epoch [13/50], Loss: 0.0903, Accuracy: 97.15%
Epoch [14/50], Loss: 0.0870, Accuracy: 97.19%
Epoch [15/50], Loss: 0.0850, Accuracy: 97.25%
Epoch [16/50], Loss: 0.0826, Accuracy: 97.26%
Epoch [17/50], Loss: 0.0770, Accuracy: 97.52%
Epoch [18/50], Loss: 0.0749, Accuracy: 97.65%
Epoch [19/50], Loss: 0.0783, Accuracy: 97.41%
Epoch [20/50], Loss: 0.0709, Accuracy: 97.65%
Epoch [21/50], Loss: 0.0722, Accuracy: 97.69%
Epoch [22/50], Loss: 0.0692, Accuracy: 97.7

## Data
- cơ sở dữ liệu MNIST tiếng Anh là MNIST database, viết tắt từ Modified National Institude of Standards and Technology. 
- là cơ sở dữ liệu biết tay thường dùng trong việc huấn luyện các hệ thống xử lí hình ảnh khác nhau.
- Nó dùng để huấn luyện và kiểm thử trong lĩnh vực học máy.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# ==== 1. LOAD DATA (Thay vì dùng dữ liệu ngẫu nhiên) ====
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ==== 2. DEFINE MODEL ====
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.3)  # Tránh overfitting

    def forward(self, x):
        x = x.view(-1, 784)  # Chuyển từ 28x28 thành vector 784
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Không dùng Softmax ở đây vì CrossEntropyLoss đã xử lý
        return x

# ==== 3. INITIALIZE MODEL, LOSS, AND OPTIMIZER ====
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam tốt hơn SGD ở đây

# ==== 4. TRAIN MODEL ====
n_epochs = 50  # Tăng số epochs để đảm bảo học tốt
train_start_time = time.time()
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
    
    print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
train_end_time = time.time()

print("Mô hình đã được huấn luyện!")

# ==== 5. EVALUATE MODEL ====
model.eval()
test_start_time = time.time()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1)  # Áp dụng Softmax để lấy xác suất
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
test_end_time = time.time()

accuracy = 100 * correct / total

# ==== 6. PRINT RESULTS ====
print(f"Độ chính xác: {accuracy:.2f}%")
print(f"Thời gian huấn luyện: {train_end_time - train_start_time:.4f} giây")
print(f"Thời gian đánh giá: {test_end_time - test_start_time:.4f} giây")
print(f"Thời gian tổng: {train_end_time - train_start_time + test_end_time - test_start_time:.4f} giây")

# ==== 7. SHOW SAMPLE PREDICTIONS ====
images, labels = next(iter(test_loader))
with torch.no_grad():
    outputs = model(images)
    probabilities = F.softmax(outputs, dim=1)  # Áp dụng Softmax để lấy xác suất

for i in range(5):
    print(f"Mẫu {i+1}:")
    print(f"- Nhãn thực tế: {labels[i].item()}")
    print(f"- Xác suất dự đoán: {probabilities[i].tolist()}\n")





Epoch [1/50], Loss: 0.3900, Accuracy: 87.90%
Epoch [2/50], Loss: 0.2089, Accuracy: 93.51%
Epoch [3/50], Loss: 0.1698, Accuracy: 94.78%
Epoch [4/50], Loss: 0.1495, Accuracy: 95.33%
Epoch [5/50], Loss: 0.1365, Accuracy: 95.69%
Epoch [6/50], Loss: 0.1226, Accuracy: 96.03%
Epoch [7/50], Loss: 0.1166, Accuracy: 96.39%
Epoch [8/50], Loss: 0.1091, Accuracy: 96.44%
Epoch [9/50], Loss: 0.1020, Accuracy: 96.67%
Epoch [10/50], Loss: 0.0989, Accuracy: 96.81%
Epoch [11/50], Loss: 0.0953, Accuracy: 97.02%
Epoch [12/50], Loss: 0.0893, Accuracy: 97.11%
Epoch [13/50], Loss: 0.0858, Accuracy: 97.27%
Epoch [14/50], Loss: 0.0846, Accuracy: 97.22%
Epoch [15/50], Loss: 0.0834, Accuracy: 97.32%
Epoch [16/50], Loss: 0.0782, Accuracy: 97.49%
Epoch [17/50], Loss: 0.0732, Accuracy: 97.68%
Epoch [18/50], Loss: 0.0736, Accuracy: 97.62%
Epoch [19/50], Loss: 0.0709, Accuracy: 97.68%
Epoch [20/50], Loss: 0.0754, Accuracy: 97.54%
Epoch [21/50], Loss: 0.0669, Accuracy: 97.80%
Epoch [22/50], Loss: 0.0676, Accuracy: 97.8